In [2]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-13 13:53:24--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

     0K ...                                                   100%  579K=0.006s

2024-06-13 13:53:24 (579 KB/s) - 'minsearch.py' saved [3832/3832]



In [1]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [2]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [7]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [12]:
rag("I just discovered the course. Can I still join in?")

' Yes, as per the context from FAQs regarding course enrollment after starting date: "Yes, even if you don\'t register, you\'re still eligible to submit the homeworks." However, be aware of deadlines for submitting final projects and try not to leave everything till the last minute.\n\nYou may also like this additional info on course start time as it is mentioned elsewhere in FAQs: The exact day and hour of the course will be 15th Jan 2024 at 17h00, starting with an "Office Hours\'\' live session for those who join beforehand.\n\nNote that there are ways to get help during self-paced mode like using slack channel or tagging bot @ZoomcampQABot but it\'s suggested always to check the FAQ document first and second as they contain most of questions already answered herein this context.'

In [13]:
print(_)

 Yes, as per the context from FAQs regarding course enrollment after starting date: "Yes, even if you don't register, you're still eligible to submit the homeworks." However, be aware of deadlines for submitting final projects and try not to leave everything till the last minute.

You may also like this additional info on course start time as it is mentioned elsewhere in FAQs: The exact day and hour of the course will be 15th Jan 2024 at 17h00, starting with an "Office Hours'' live session for those who join beforehand.

Note that there are ways to get help during self-paced mode like using slack channel or tagging bot @ZoomcampQABot but it's suggested always to check the FAQ document first and second as they contain most of questions already answered herein this context.


In [11]:
llm('hi?')

" Hello! I'm an AI developed to assist with information, tasks, and general knowledge. How can I help you today? If there are specific questions or things on your mind, feel free to share them here—whether it be for clarification, learning a new fact, or solving a problem!"